# TensorFlow Neural Network for XOR

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
xy = np.loadtxt('train_nn_xor.txt', unpack=False)
xy

array([[ 0.,  0.,  0.],
       [ 0.,  1.,  1.],
       [ 1.,  0.,  1.],
       [ 1.,  1.,  0.]])

In [3]:
x_data = xy[:, 0:-1]
x_data

array([[ 0.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  1.]])

In [4]:
print(len(x_data))
print(len(x_data[0]))

4
2


In [5]:
y_data = xy[:, -1]
y_data

array([ 0.,  1.,  1.,  0.])

In [6]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

## 먼저 Logistic Regression  으로 작업해 본다면

In [7]:
W = tf.Variable(tf.random_uniform([len(x_data[0]), 1], -1.0, 1.0))

In [8]:
h = tf.matmul(X, W)

In [9]:
hypo = tf.div(1. , 1. + tf.exp(-h))
# hypo = tf.sigmoid(h) 를 풀어쓰면 위와 같습니다.

In [10]:
cost = -tf.reduce_mean(Y*tf.log(hypo) + (1-Y)*tf.log(1-hypo))

In [11]:
alpha = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(alpha)
train = optimizer.minimize(cost)

In [12]:
init = tf.initialize_all_variables()

In [13]:
sess = tf.Session()
sess.run(init)

In [14]:
for step in range(1000):
    sess.run(train, feed_dict={X:x_data, Y:y_data})
    if step % 200 == 0:
        print("step : ", step)
        print("cost : ", sess.run(cost, feed_dict={X:x_data, Y:y_data}))
        print(sess.run(W))
        print("="*50)


step :  0
cost :  0.743497
[[-0.42094618]
 [-0.62607086]]
step :  200
cost :  0.693231
[[ 0.01731055]
 [-0.04194125]]
step :  400
cost :  0.693152
[[ 0.00817555]
 [-0.00873461]]
step :  600
cost :  0.693148
[[ 0.00240661]
 [-0.00241929]]
step :  800
cost :  0.693147
[[ 0.00068848]
 [-0.00068876]]


In [15]:
corrent_prediction = tf.equal(tf.floor(hypo + 0.5), Y)

In [16]:
accuracy = tf.reduce_mean(tf.cast(corrent_prediction, 'float'))

In [17]:
print("hypo : \n", sess.run(hypo, feed_dict={X:x_data, Y:y_data}))
print("floor : \n", sess.run(tf.floor(hypo+0.5), feed_dict={X:x_data, Y:y_data}))
print("corrent_prediction : \n", sess.run(corrent_prediction, feed_dict={X:x_data, Y:y_data}))
print("accuracy : \n", sess.run(accuracy, feed_dict={X:x_data, Y:y_data}))

hypo : 
 [[ 0.5       ]
 [ 0.49995053]
 [ 0.50004941]
 [ 0.5       ]]
floor : 
 [[ 1.]
 [ 0.]
 [ 1.]
 [ 1.]]
corrent_prediction : 
 [[False  True  True False]
 [ True False False  True]
 [False  True  True False]
 [False  True  True False]]
accuracy : 
 0.5


## 결론 : Logistic Regression 으로는 이론적으로도 실질적으로도 0.5 이상 나올 수가 없다.

## 그렇다면, Layer 를 한개 더 둠으로써, NN 을 이용해서 구해보자